In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':''})
downloaded.GetContentFile('train.csv')

In [4]:
import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt
plt.switch_backend('agg')

Using TensorFlow backend.


In [0]:
class GAN(object):
  def __init__(self, width = 28, height = 28, channels = 1):
    
    self.WIDTH = width
    self.HEIGHT = height
    self.CHANNELS = channels
    
    self.SHAPE = (self.WIDTH, self.HEIGHT, self.CHANNELS)
    
    self.OPTIMIZER = Adam(lr = 0.0002, decay = 8e-9)
    
    self.noise_gen = np.random.normal(0, 1, (100, ))
    
    self.G = self.generator()
    
    self.G.compile(loss = 'binary_crossentropy', optimizer = self.OPTIMIZER)
    
    self.D = self.discriminator()
    
    self.D.compile(loss = 'binary_crossentropy', optimizer = self.OPTIMIZER, metrics = ['accuracy'])
    
    self.stacked_G_D = self.stacked_G_D()
    
    self.stacked_G_D.compile(loss = 'binary_crossentropy', optimizer = self.OPTIMIZER)
  
  def generator(self):
    
    model = Sequential()
    
    model.add(Dense(256, input_shape = (100,)))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(self.WIDTH * self.HEIGHT * self.CHANNELS, activation = 'tanh'))
    
    model.add(Reshape((self.WIDTH, self.HEIGHT, self.CHANNELS)))
    
    return model
  
  def discriminator(self):

    model = Sequential()
    model.add(Flatten(input_shape=self.SHAPE))
    model.add(Dense((self.WIDTH * self.HEIGHT * self.CHANNELS), input_shape=self.SHAPE))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(int((self.WIDTH * self.HEIGHT * self.CHANNELS)/2)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    return model
  
  def stacked_G_D(self):
    self.D.trainable = False
    
    model = Sequential()
    model.add(self.G)
    model.add(self.D)
    
    return model
  
  def train(self, X_train, epochs = 20000, batch = 32, save_interval = 200):
    for cnt in range(epochs):
      random_index = np.random.randint(0, len(X_train) - int(batch/2))
      legit_images = X_train[random_index : random_index + int(batch/2)].reshape(int(batch/2), self.WIDTH, self.HEIGHT, self.CHANNELS)
      
      gen_noise = np.random.normal(0, 1, (int(batch/2), 100))
      synthetic_images = self.G.predict(gen_noise)
      
      x_combined_batch = np.concatenate((legit_images, synthetic_images))
      y_combined_batch = np.concatenate((np.ones((int(batch/2), 1)), np.zeros((int(batch/2), 1))))
      
      d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)
      
      noise = np.random.normal(0, 1, (batch, 100))
      y_mislabeled = np.ones((batch, 1))
      
      g_loss = self.stacked_G_D.train_on_batch(noise, y_mislabeled)
      
      print('epoch: %d, [Discriminator :: d_loss: %f], [Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))
      
      if cnt % save_interval == 0:
        self.plot_images(save2file = True, step = cnt)
        
  def plot_images(self, save2file =False, samples =16, step=0):
    filename = "mnist_%d.png" % step
    noise = np.random.normal(0, 1, (samples, 100))
    
    images = self.G.predict(noise)
    
    plt.figure(figsize = (10,10))
    
    for i in range(images.shape[0]):
      plt.subplot(4, 4, i+1)
      image = images[i, :, :, :]
      image = np.reshape(image, [ self.HEIGHT, self.WIDTH ])
      plt.imshow(image, cmap='gray')
      plt.axis('off')
      plt.tight_layout()
      
    if save2file:
      plt.savefig(filename)
      plt.close('all')
    else:
      plt.show()

In [0]:
import pandas as pd

In [10]:
train = pd.read_csv('train.csv')

print(train.columns)
train.head()

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
len(train)

42000

In [0]:
X_train = train.iloc[:,1:].values

In [0]:
xtrain = []
for i in range(len(X_train)):
  xtrain.append(X_train[i].reshape(28,28,1))

In [0]:
X_train = np.array(xtrain)

In [48]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

gan = GAN()

gan.train(X_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 784)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 392)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 393       
Total params: 923,553
Trainable params: 923,553
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.828876], [Generator :: loss: 0.805515]
epoch: 1, [Discriminator :: d_loss: 0.380148], [Generator :: loss: 0.863493]
epoch: 2, [Discriminator :: d_loss: 0.280274], [Generator :: loss: 0.959760]
epoch: 3, [Discriminator :: d_loss: 0.337523], [Generator :: loss: 1.083287]
epoch: 4, [Discriminator :: d_loss: 0.209875], [Generator :: loss: 1.183381]
epoch: 5, [Discriminator :: d_loss: 0.192510], [Generator :: loss: 1.347799]
epoch: 6, [Discriminator :: d_loss: 0.158964], [Generator :: loss: 1.527394]
epoch: 7, [Discriminator :: d_loss: 0.160406], [Generator :: loss: 1.622852]
epoch: 8, [Discriminator :: d_loss: 0.123795], [Generator :: loss: 1.904137]
epoch: 9, [Discriminator :: d_loss: 0.091743], [Generator :: loss: 2.041026]
epoch: 10, [Discriminator :: d_loss: 0.089119], [Generator :: loss: 2.247911]
epoch: 11, [Discriminator :: d_loss: 0.072038], [Generator :: loss: 2.394490]
epoch: 12, [Discriminator :: d_loss: 0.062810], [Generator :: loss: 2.5889